# Uebung 3

## Aufgabe 1

a) Laden Sie das tiff-file des 0.1m swissimage Datensatz herunter ("daten/bern.tif"), welches das Bundeshaus enthält. 

b) Erstellen Sie ein neues GeoTIFF im Umkreis von 100 Metern um das Bundeshaus (1000x1000 Bild).


Das Bundeshaus ist hier zu finden:

    pos_lv95 = (2_600_426.281, 1_199_497.112) 
    pos_wgs84 = (7.444120726365559, 46.94653998135123)

In [4]:
# a)

import geopandas as gpd
import shapely
import geopandas_stac as stac
import geoutils

stac.getCollectionList(cache=True)
df = stac.getAssets("ch.swisstopo.swissimage-dop10")

df_swissimage_10cm = df.query('gsd == "0.1"') 

lat = 46.94653998135123
lng = 7.444120726365559

point = shapely.geometry.Point(lng, lat)
bundeshaus_punkt = gpd.GeoDataFrame(geometry=gpd.GeoSeries(point, crs="epsg:4326"))

df_bundeshaus_10cm = gpd.sjoin(df_swissimage_10cm, bundeshaus_punkt, op='contains')
bundeshaus = stac.getUrlList(df_bundeshaus_10cm)
url = bundeshaus[0]

geoutils.download(url, "daten/bern.tif", False)

File https://data.geo.admin.ch/ch.swisstopo.swissimage-dop10/swissimage-dop10_2018_2600-1199/swissimage-dop10_2018_2600-1199_0.1_2056.tif already exists, not overwriting.          

In [5]:
import rasterio
from rasterio.windows import Window

dataset = rasterio.open("daten/bern.tif")

# Zentrum in Pixelkoordinaten bestimmen:
pos = (2_600_426.281, 1_199_497.112)
center = ~dataset.transform * pos

# Koordinate oben links berechnen

width = 1000  # 100m -> Eigentlich Umkreis -> Pythagoras
height = 1000 # 100m    Aber nehmen wir doch 1000x1000
upper_left = (int(center[0] - width/2), int(center[1] - height/2))

w = Window(upper_left[0], upper_left[1], width, height)

r = dataset.read(1, window=w)
g = dataset.read(2, window=w)
b = dataset.read(3, window=w)

# Erstellen neuer Affintransformation
ul = dataset.transform * upper_left
t = rasterio.Affine(dataset.transform[0],dataset.transform[1],ul[0],
                    dataset.transform[3],dataset.transform[4],ul[1])

dataset_out =  rasterio.open("daten/bundeshaus.tif", "w", driver='GTiff', width=width, height=height, count=3, dtype=r.dtype, transform=t)

# Daten (r,g,b) schreiben
dataset_out.write(r, indexes=1)
dataset_out.write(g, indexes=2)
dataset_out.write(b, indexes=3)


# schliessen:
dataset_out.close()
dataset.close()

# Speicher freigeben
r = 0
g = 0
b = 0


## Aufgabe 2

Machen Sie dassebe wie Aufgabe 1, aber mit dem swisstopo WMS & Python  

In [6]:
from owslib.wms import WebMapService

def getImageFromCenter(wms,layer,centerx,centery,meters,res_cm,output_file):
    ulx = centerx-meters/2
    lrx = centerx+meters/2
    uly = centery+meters/2
    lry = centery-meters/2

    w = int((lrx-ulx)/res_cm)
    h = int((uly-lry)/res_cm)
    
    img = wms.getmap(layers=[layer],
                 styles=['default'],
                 srs='EPSG:2056',
                 bbox=(ulx, lry, 
                       lrx, uly),
                 size=(w, h),
                 format='image/tiff',
                 transparent=True)
    
    out = open(output_file, 'wb')
    out.write(img.read())
    out.close()

wms = WebMapService('https://wms.geo.admin.ch', version='1.3.0')

layer = 'ch.swisstopo.swissimage'
getImageFromCenter(wms, layer, 2_600_426.281, 1_199_497.112, 100, 10, "daten/bundeshaus_wms.tif")


/opt/anaconda3/envs/geopython/lib/python3.7/site-packages/owslib/map/wms130.py:122: UserWarning: Content metadata for layer "ch.bakom.notruf-118_mobilnetz" already exists. Using child layer
  warnings.warn('Content metadata for layer "%s" already exists. Using child layer' % cm.id)


## Aufgabe 3

Erstellen Sie einige Bild von anderen WMS Layern des swisstopo WMS.

In [7]:
layer = 'ch.swisstopo.landeskarte-farbe-10'
img = wms.getmap(layers=[layer],
                 styles=['default'],
                 srs='EPSG:2056',
                 bbox=(2_600_000, 1_199_000, 
                       2_601_000, 1_200_000),
                 size=(1000, 1000),
                 format='image/png',
                 transparent=True)
    
out = open("daten/pk.png", 'wb')
out.write(img.read())
out.close()

In [8]:
layer = 'ch.swisstopo.swissnames3d_label'
img = wms.getmap(layers=[layer],
                 styles=['default'],
                 srs='EPSG:2056',
                 bbox=(2_600_000, 1_199_000, 
                       2_601_000, 1_200_000),
                 size=(1000, 1000),
                 format='image/png',
                 transparent=True)
    
out = open("daten/sa.png", 'wb')
out.write(img.read())
out.close()